# Q1. Embedding the query

In [1]:
from fastembed import TextEmbedding
import numpy as np

/Users/shayanshafieemoghadam/personal/llm-zoomcamp-playground/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
query = 'I just discovered the course. Can I join now?'

In [28]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
model = next((m for m in TextEmbedding.list_supported_models() if m["model"] == model_handle), None)
model

{'model': 'jinaai/jina-embeddings-v2-small-en',
 'sources': {'hf': 'xenova/jina-embeddings-v2-small-en',
  'url': None,
  '_deprecated_tar_struct': False},
 'model_file': 'onnx/model.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 8192 input tokens truncation, Prefixes for queries/documents: not necessary, 2023 year.',
 'license': 'apache-2.0',
 'size_in_GB': 0.12,
 'additional_files': [],
 'dim': 512,
 'tasks': {}}

In [ ]:
q = list(TextEmbedding(model_name=model_handle).embed(query))[0]

In [46]:
min(q)

np.float64(-0.11726373551188797)

In [47]:
len(q)

512

# Q2. Cosine similarity with another vector

In [48]:
np.linalg.norm(q)

np.float64(1.0)

In [49]:
doc = 'Can I still join the course after the start date?'
d = list(TextEmbedding(model_name=model_handle).embed(doc))[0]

In [50]:
q.dot(d)

np.float64(0.9008528856818037)

# Q3. Ranking by cosine

In [51]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [58]:
for i, doc in enumerate(documents):
    d = list(TextEmbedding(model_name=model_handle).embed(doc["text"]))[0]
    print(i, q.dot(d))

0 0.7629684493123693
1 0.8182378325403556
2 0.8085397290762828
3 0.713307860072464
4 0.7304499388234831


In [59]:
docs = [doc["text"] for doc in documents]

In [71]:
D = list(TextEmbedding(model_name=model_handle).embed(docs))

In [72]:
np.shape(D)

(5, 512)

In [75]:
np.shape(q)

(512,)

In [77]:
np.dot(D, q)

array([0.76296845, 0.81823784, 0.80853973, 0.71330785, 0.73044995])

# Q4. Ranking by cosine, version two

In [80]:
new_docs = [doc["question"] + " " + doc["text"] for doc in documents]
D = list(TextEmbedding(model_name=model_handle).embed(new_docs))

In [81]:
np.dot(D, q)

array([0.8514543 , 0.84365941, 0.84082872, 0.77551577, 0.80860079])

# Q5. Selecting the embedding model

In [2]:
dims = [m["dim"] for m in TextEmbedding.list_supported_models()]
min(dims)

384

In [3]:
model_handle  = 'BAAI/bge-small-en'
model = next((m for m in TextEmbedding.list_supported_models() if m["model"] == model_handle), None)
model["dim"]

384

# Q6. Indexing with qdrant (2 points)

```bash
docker run --rm -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/.qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant
```

In [4]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
from qdrant_client import QdrantClient, models

In [6]:
client = QdrantClient("http://localhost:6333")

In [7]:
model_handle  = 'BAAI/bge-small-en'
collection_name = "ml-zoomcamp-rag"
embedding_dim = model["dim"]

In [ ]:
client.create_coxllection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=embedding_dim,
        distance=models.Distance.COSINE
    )
)

True

In [8]:
points = []
for i, doc in enumerate(documents):
    points.append(models.PointStruct(
            id=i,
            vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_handle),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            }
        ))

In [9]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:  20%|██        | 1/5 [00:10<00:40, 10.00s/it]
2025-06-28 18:05:45.751 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: An error happened while trying to locate the file on the Hub and we cannot find the requested files in the local cache. Please check your connection and try again or make sure your Internet connection is on. Falling back to other sources.
100%|██████████| 77.7M/77.7M [01:11<00:00, 1.08MiB/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
result = client.query_points(
    collection_name=collection_name,
    query=models.Document(text=query, model=model_handle)
)

In [18]:
result.points[0].score

0.8703172